In [12]:
import mlflow
import dagshub
dagshub.init(repo_owner='tquangbk', repo_name='mlflow-master', mlflow=True)

Initialized MLflow to track repo "tquangbk/mlflow-master"

Repository tquangbk/mlflow-master initialized!

# Load a specific model from DagsHub, then run local inference

In [14]:
logged_model = 'runs:/9bc90c62322043dbb18ee47c39e1f36b/model'
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
print("loaded model:", loaded_model)

# Predict on a Pandas DataFrame.
import pandas as pd
# Create the JSON payload
data = {
    "inputs": [[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]]
}
loaded_model.predict(pd.DataFrame(data))

loaded model: mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 9bc90c62322043dbb18ee47c39e1f36b



array([0.42419148])

In [4]:
# !mlflow models serve -m runs:/9bc90c62322043dbb18ee47c39e1f36b/model -p 5000

^C


# List mlflow experiments

In [15]:
!mlflow runs list --experiment-id 0

Date                                       Name                      ID                              
-----------------------------------------  ------------------------  --------------------------------
2024-10-07 10:06:34 Romance Daylight Time  lr_base_3                 9bc90c62322043dbb18ee47c39e1f36b
2024-10-04 23:15:18 Romance Daylight Time  lr-base                   e32bad255b0d4c43a15ba95753a2bacf
2024-10-04 22:54:39 Romance Daylight Time  lr-base                   23891785c95f4a95b8cf1b6f50662ab0
2024-10-04 21:36:58 Romance Daylight Time  sincere-hen-89            a8cee4a2422b4a7c913f2c864c2892b9
2024-10-04 21:36:58 Romance Daylight Time  sedate-pig-733            a31145d41e3e45c797159c57eb0f5508
2024-10-04 21:36:58 Romance Daylight Time  lr-hyperparameter-tuning  03137bfcf664457c98b344507c8544a9
2024-10-04 21:36:58 Romance Daylight Time  legendary-mole-678        744b17c09d1643d89c0b72403a9c510a
2024-10-04 21:36:58 Romance Daylight Time  dapper-newt-309           4c26e26901eb4

# Searching mlflow Models

## Fetch all registered models

In [16]:
# create a client to access the MLflow tracking server
client = mlflow.MlflowClient()

# loop through all registered models
# NOTE: `filter_string` should be optional, but leaving it as `None` failed to work. 
# Instead, using `"name LIKE '%'"` will match all model names
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    # loop through the latest versions for each stage of a registered model
    for model_version in model.latest_versions:
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}, alias={model_version.aliases}")

name=best_lr; run_id=03137bfcf664457c98b344507c8544a9; version=1, stage=None, alias=[]
name=lr_base; run_id=23891785c95f4a95b8cf1b6f50662ab0; version=1, stage=None, alias=[]
name=lr_base_2; run_id=e32bad255b0d4c43a15ba95753a2bacf; version=1, stage=None, alias=[]
name=lr_base_3; run_id=9bc90c62322043dbb18ee47c39e1f36b; version=1, stage=None, alias=[]


## Query specific registered models

In [22]:
client = mlflow.MlflowClient()

# loop through any registered model, whose name starts with `Cool`
for model in client.search_registered_models(filter_string="name LIKE 'best%'"):
    for model_version in model.latest_versions:
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}, alias={model_version.aliases}")

name=best_lr; run_id=03137bfcf664457c98b344507c8544a9; version=1, stage=None, alias=[]


# Update Model Information

## Rename a Model

In [25]:
# create a client to access the MLflow tracking server
client = mlflow.MlflowClient()

# rename the model from `Ultimate Mario Model` to `Great Mario Model`
client.rename_registered_model(
    name='lr_base_2', 
    new_name='lr_base_2 beginner'
)

# loop through all registered models
# NOTE: `filter_string` should be optional, but leaving it as `None` failed to work. 
# Instead, using `"name LIKE '%'"` will match all model names
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    # loop through the latest versions for each stage of a registered model
    for model_version in model.latest_versions:
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}, alias={model_version.aliases}")

name=best_lr; run_id=03137bfcf664457c98b344507c8544a9; version=1, stage=None, alias=[]
name=lr_base; run_id=23891785c95f4a95b8cf1b6f50662ab0; version=1, stage=None, alias=[]
name=lr_base_2 beginner; run_id=e32bad255b0d4c43a15ba95753a2bacf; version=1, stage=None, alias=[]
name=lr_base_3 beginner; run_id=9bc90c62322043dbb18ee47c39e1f36b; version=1, stage=None, alias=[]


## Update Stage

In [26]:
# create a client to access the MLflow tracking server
client = mlflow.MlflowClient()

# transition version 1 of the `Great Mario Model` model from `None` to `Staging`
client.transition_model_version_stage(
    name='lr_base',
    version=1,
    stage="Staging"
)
# log all models
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    # loop through the latest versions for each stage of a registered model
    for model_version in model.latest_versions:
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}, alias={model_version.aliases}")

C:\Users\easyb\AppData\Local\Temp\ipykernel_28700\3356450396.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


name=best_lr; run_id=03137bfcf664457c98b344507c8544a9; version=1, stage=None, alias=[]
name=lr_base; run_id=23891785c95f4a95b8cf1b6f50662ab0; version=1, stage=Staging, alias=[]
name=lr_base_2 beginner; run_id=e32bad255b0d4c43a15ba95753a2bacf; version=1, stage=None, alias=[]
name=lr_base_3 beginner; run_id=9bc90c62322043dbb18ee47c39e1f36b; version=1, stage=None, alias=[]


In [27]:
# log all models
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    # loop through the latest versions for each stage of a registered model
    for model_version in model.latest_versions:
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}, alias={model_version.aliases}")

name=best_lr; run_id=03137bfcf664457c98b344507c8544a9; version=1, stage=None, alias=[]
name=lr_base; run_id=23891785c95f4a95b8cf1b6f50662ab0; version=1, stage=Staging, alias=[]
name=lr_base_2 beginner; run_id=e32bad255b0d4c43a15ba95753a2bacf; version=1, stage=None, alias=[]
name=lr_base_3 beginner; run_id=9bc90c62322043dbb18ee47c39e1f36b; version=1, stage=None, alias=[]


## Loading Models

In [29]:
# Model name
name = 'best_lr'

# Get the latest version for the model
version = client.get_latest_versions(name=name)[0].version

# Construct the model URI
model_uri = f'models:/{name}/{version}'

# Load the model
loaded_model_2 = mlflow.sklearn.load_model(model_uri)

C:\Users\easyb\AppData\Local\Temp\ipykernel_28700\247662276.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  version = client.get_latest_versions(name=name)[0].version


In [33]:
loaded_model_2

ElasticNet(alpha=0.04435572858845327, l1_ratio=0.1203260806412767)

In [38]:
# Depending on how the model is loaded, the data formated should be adapted
data = [[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]]
loaded_model_2.predict(data)

array([-0.06795475])

## Saving our changes

In [39]:
!git commit -a -m "added model registering"
!git push origin main

[main 93054dd] added model registering
 4 files changed, 770 insertions(+), 45 deletions(-)
 delete mode 100644 mlruns/0/meta.yaml


To https://dagshub.com/tquangbk/mlflow-master.git
   db948f2..93054dd  main -> main


In [40]:
loaded_model_2

ElasticNet(alpha=0.04435572858845327, l1_ratio=0.1203260806412767)

## Building and Serving with Docker Images

In [48]:
# Set the MLflow tracking URI to your DagsHub repository
import os

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/tquangbk/mlflow-master.mlflow'

# Build the Docker image using the registered model URI
!mlflow models build-docker --name lr --model-uri "models:/best_lr/1"

2024/10/07 11:48:06 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/10/07 11:48:07 INFO mlflow.pyfunc.backend: Building docker image with name lr
#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.57kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:20.04
#3 DONE 0.5s

#4 [ 1/17] FROM docker.io/library/ubuntu:20.04@sha256:6d8d9799fe6ab3221965efac00b4c34a2bcc102c086a58dff9e19a08b913c7ef
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 2.27kB done
#5 DONE 0.0s

#6 [14/17] COPY model_dir/ /opt/ml/model
#6 CACHED

#7 [12/17] RUN cp /opt/java/mlflow-scoring-2.16.2.pom /opt/java/pom.xml
#7 CACHED

#8 [13/17] RUN cd /opt/java && mvn --batch-mode dependency:copy-dependencies -DoutputDirectory=/opt/java/jars
#8 CACHED

#9 [15/17] RUN python -c "from mlflow.models import container as C; C._

In [57]:
# Set the MLflow tracking URI to your DagsHub repository
import os

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/tquangbk/mlflow-master.mlflow'

# Build the Docker image using the registered model URI
!mlflow models build-docker --name lr-test-2 -m "runs:/9bc90c62322043dbb18ee47c39e1f36b/model"

2024/10/07 12:29:45 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/10/07 12:29:46 INFO mlflow.pyfunc.backend: Building docker image with name lr-test-2
#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.57kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:20.04
#3 DONE 0.6s

#4 [ 1/17] FROM docker.io/library/ubuntu:20.04@sha256:6d8d9799fe6ab3221965efac00b4c34a2bcc102c086a58dff9e19a08b913c7ef
#4 DONE 0.0s

#5 [ 6/17] RUN pip install virtualenv
#5 CACHED

#6 [ 5/17] RUN apt install -y python3.8 python3.8-distutils     && ln -s -f $(which python3.8) /usr/bin/python     && wget https://bootstrap.pypa.io/get-pip.py -O /tmp/get-pip.py     && python /tmp/get-pip.py
#6 CACHED

#7 [12/17] RUN cp /opt/java/mlflow-scoring-2.16.2.pom /opt/java/pom.xml
#7 CACHED

#8 [ 9/17] RUN pip install mlflow==2.16.2
#8 CACHED


In [59]:
!pip install mlflow[mlserver]

In [62]:
# maybe does not work with dagshub
# !mlflow models serve -m runs:/9bc90c62322043dbb18ee47c39e1f36b/model -p 5000

# !mlflow models serve -m "models:/best_lr/1" -p 5000

^C
